In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

# **Classificação de Currículos**
Pré-seleção e triagem inicial de candidatos em processos seletivos.

In [3]:
llm_curriculos = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
CURRICULO_PROMPT = (
    "Você é um triador de currículos para a vaga de 'Desenvolvedor Python Júnior'. "
    "Analise o texto e extraia as seguintes informações no formato JSON:\n\n"

    # Exemplo 1 (Candidato Júnior)
    "Currículo: João da Silva. Desenvolvedor Python com 1 ano de experiência. Conhecimentos em Django, SQL e Git. Busco minha primeira oportunidade como júnior.\n"
    "JSON: {\n"
    "  \"nivel_senioridade\": \"JUNIOR\",\n"
    "  \"anos_experiencia\": 1,\n"
    "  \"habilidades_tecnicas\": [\"Django\", \"SQL\", \"Git\"],\n"
    "  \"nota_compatibilidade\": 85\n"
    "}\n\n"

    # Exemplo 2 (Candidato Sênior)
    "Currículo: Maria Oliveira. Engenheira de Software com mais de 10 anos de experiência em Python. Especialista em microserviços com Flask e Docker. Liderou times de backend em projetos de grande escala.\n"
    "JSON: {\n"
    "  \"nivel_senioridade\": \"SENIOR\",\n"
    "  \"anos_experiencia\": 10,\n"
    "  \"habilidades_tecnicas\": [\"Python\", \"Flask\", \"Docker\", \"microserviços\"],\n"
    "  \"nota_compatibilidade\": 60\n"
    "}\n\n"

    # Exemplo 3 (Candidato Pleno)
    "Currículo: Pedro Costa. Desenvolvedor Python com 4 anos de experiência. Proficiência em Django, FastAPI e APIs REST. Certificação AWS. Atuei em projetos de médio porte.\n"
    "JSON: {\n"
    "  \"nivel_senioridade\": \"PLENO\",\n"
    "  \"anos_experiencia\": 4,\n"
    "  \"habilidades_tecnicas\": [\"Django\", \"FastAPI\", \"APIs REST\", \"AWS\"],\n"
    "  \"nota_compatibilidade\": 95\n"
    "}\n\n"

    "Currículo: {curriculo}\n"
    "JSON: "

)

In [5]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class CandidatoOut(BaseModel):
  nivel_senioridade: Literal['JUNIOR', 'PLENO', 'SENIOR']
  anos_experiencia: int
  habilidades_tecnicas: List[str] = Field(default_factory=list)
  nota_compatibilidade: int

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage

curriculo_chain = llm_curriculos.with_structured_output(CandidatoOut)

def triagem_curriculo(curriculo: str) -> Dict:
  saida: CandidatoOut = curriculo_chain.invoke([
      SystemMessage(content=CURRICULO_PROMPT),
      HumanMessage(content=curriculo)
  ])

  return saida.model_dump()

In [8]:
testes = [
    "Ana Souza. Recém-formada em Ciência da Computação. Possui 8 meses de experiência em estágio com desenvolvimento backend. Tem sólidos conhecimentos em Python, Pandas, APIs REST e SQL. Entusiasmada em iniciar a carreira como Desenvolvedora Júnior.",
    "Lucas Ferreira. Desenvolvedor Full-stack com 5 anos de experiência. Proficiência em Python, Django, Docker, e AWS. Liderou projetos de médio porte e participou de todo o ciclo de vida do desenvolvimento de software.",
    "Camila Santos. Arquiteta de Software. Vasta experiência com mais de 12 anos em projetos de larga escala. Especialista em arquiteturas de microsserviços e sistemas distribuídos. Experiência em liderança técnica e mentoring de equipes.",
    "Fernanda Lima. Recém-formada em Engenharia de Software. Participou de um projeto de robótica usando Python, mas sem experiência com bancos de dados. Conhece apenas a sintaxe básica de Python e Git.",
    "Ricardo Alves. Profissional com grande experiência em desenvolvimento web. Trabalhou com Python e Flask. Fez um curso avançado de Docker e Kubernetes."
]

In [10]:
for curriculo in testes:
  print(f"Currículo: {curriculo}\nResultado do candidato: {triagem_curriculo(curriculo)}\n")

Currículo: Ana Souza. Recém-formada em Ciência da Computação. Possui 8 meses de experiência em estágio com desenvolvimento backend. Tem sólidos conhecimentos em Python, Pandas, APIs REST e SQL. Entusiasmada em iniciar a carreira como Desenvolvedora Júnior.
Resultado do candidato: {'nivel_senioridade': 'JUNIOR', 'anos_experiencia': 0, 'habilidades_tecnicas': ['Python', 'Pandas', 'APIs REST', 'SQL'], 'nota_compatibilidade': 90}

Currículo: Lucas Ferreira. Desenvolvedor Full-stack com 5 anos de experiência. Proficiência em Python, Django, Docker, e AWS. Liderou projetos de médio porte e participou de todo o ciclo de vida do desenvolvimento de software.
Resultado do candidato: {'nivel_senioridade': 'PLENO', 'anos_experiencia': 5, 'habilidades_tecnicas': ['Python', 'Django', 'Docker', 'AWS'], 'nota_compatibilidade': 55}

Currículo: Camila Santos. Arquiteta de Software. Vasta experiência com mais de 12 anos em projetos de larga escala. Especialista em arquiteturas de microsserviços e sistema